In [ ]:
# # 查看有哪些python库可用
# import pkg_resources
# installed_packages = pkg_resources.working_set
# installed_packages_list = sorted(["%s==%s" % (i.key, i.version) for i in installed_packages])
# print(installed_packages_list)
# # 查看可用python库的位置
# import pkg_resources
# installed_packages = pkg_resources.working_set
# for package in installed_packages:
#     print(package.key, ":", package.version)
#     print("Location:", package.location)
# #安装httpx包
# !/opt/conda/bin/pip3 install httpx --target=/home/jquser/tmp
# !tar -zcvf lib.tar.gz tmp


# 打印交割单
from jqdata import *
import pandas as pd
# 设置回测 ID
backtest_id = '07db2700d5a25657cd61b5b8e412f7f0'
# 获取回测报告
gt = get_backtest(backtest_id)
df=gt.get_orders()
df=pd.DataFrame(df)
df = df.rename(columns={'time': '时间','action': '开平仓', 
                        'amount': '数量', 'commission':'手续费','filled': '已成交量', 
                        'gains': '收益', 'price':'成交价','security': '标的代码', 
                        'status': '订单状态', 'type':'委托方式'})
df.to_csv(f"回测{backtest_id}.csv")
df2=gt.get_positions()
df2=pd.DataFrame(df2)
df2.to_csv(f"持仓{backtest_id}.csv")
print(df,df2)


from jqdata import *
import pandas as pd
import datetime
import time
import math
def filter_kcb_stock(stocks): # 过滤科创北交股票
    for stock in stocks[:]:
        if stock[0] == "4" or stock[0] == "8" or stock[:2] == "68":
            stocks.remove(stock)
    return stocks
now = datetime.datetime.now()
start_date = now.strftime("%Y-%m-%d")  # 测试当天的数据
# start_date = (now - datetime.timedelta(days=1)).strftime("%Y-%m-%d")  # 测试昨天的数据
today = get_trade_days(start_date=None, end_date=start_date, count=2)[1]
yesterday = get_trade_days(start_date=None, end_date=start_date, count=2)[0]
print(today,yesterday)


# #行业数据获取(聚宽)
# try:
#     pd.read_csv("【行业】聚宽一级行业对照表"+str(start_date)+".csv")
#     print("******","【行业】聚宽一级行业对照表"+str(start_date)+".csv"+ "文件存在")
# except Exception as e:
#     print("******","******" +"【行业】聚宽一级行业对照表"+str(start_date)+".csv"+ "文件不同时存在")
#     yesterday=today-datetime.timedelta(days=1)
#     yesterday=yesterday.strftime("%Y%m%d") # 回测前一天
#     stockslist=get_industries(name="jq_l1").index.tolist() # 聚宽一级行业数据(一创聚宽)
#     print(stockslist)
#     bkdf=pd.DataFrame()
#     for stock in stockslist: # 获取行业类成分股
#         stocks=get_industry_stocks(stock,date=today) # 行业成分股
# #         stocks = filter_kcb_stock(stocks) # 去除科创北交
#         df=pd.DataFrame({"代码":stocks})
#         df["板块"]=stock
#         bkdf=pd.concat([bkdf,df],axis=0,sort=False)
#     bkdf=bkdf.reset_index()
#     print("len(bkdf)",bkdf)
#     write_file("【行业】聚宽一级行业对照表"+str(start_date)+".csv",bkdf.to_csv(index=False),append=True)


#数据拼接
from jqdata import finance
thisnames=["AH股价格对比",#【AH价格对比是无效因子】
           "大股东增减持",
           "新闻联播文本",
           "融资融券数据"]
for thisname in thisnames:
    bkname=thisname
    allnum=50
    dt=pd.DataFrame({}) #初始化一个空的dt用来拼接因子数据
    daysnum=1200
    start_date=((datetime.datetime.now()-datetime.timedelta(days=daysnum)).strftime("%Y-%m-%d")) #测试当天的数据
    print(start_date)
    end_date=((datetime.datetime.now()).strftime("%Y-%m-%d")) #测试当天的数据
    tradeday=get_trade_days(start_date=start_date,end_date=end_date).tolist()
    print("start_date",start_date,"tradeday",tradeday,len(tradeday),type(tradeday),type(tradeday[0]))
    tradenum=0#用来计算当前总交易天数
    for today in tradeday:
        tradenum+=1#用来计算当前总交易天数【目的是确定是否重置仓位】
        print(today)
        index=tradeday.index(today)
        #不取后四天的数据避免越界
        if (index>=0) and (index<len(tradeday)-4):
            yesterday=tradeday[index-1]
            if thisname=="大股东增减持":
#                 df=finance.run_query(query(finance.STK_SHAREHOLDERS_SHARE_CHANGE))
                df=finance.run_query(query(finance.STK_SHAREHOLDERS_SHARE_CHANGE).filter(finance.STK_SHAREHOLDERS_SHARE_CHANGE.pub_date==today))
            elif thisname=="新闻联播文本":
                df=finance.run_query(query(finance.CCTV_NEWS).filter(finance.CCTV_NEWS.day==today))
            elif thisname=="AH股价格对比":
                df=finance.run_query(query(finance.STK_AH_PRICE_COMP).filter(finance.STK_AH_PRICE_COMP.day==today))
            elif thisname=="融资融券数据":
                stocks = list(get_all_securities(types=['stock'], date=yesterday).index)
                df=get_mtss(stocks,start_date=yesterday,end_date=today,fields=None)#融资融券数据
        else:
            print("时间结束不再获取数据")
#             break
        print(df)
        dt.to_csv(f"{thisname}.csv")
        dt=pd.concat([dt,df])
    print(df)
    dt.to_csv(f"{thisname}.csv")
    
    
    
    
# # 股东股份质押
# from jqdata import finance
# finance.run_query(query(finance.STK_SHARES_PLEDGE).filter(finance.STK_SHARES_PLEDGE.code==code).limit(n))
# #股份冻结情况
# from jqdata import finance
# finance.run_query(query(finance.STK_SHARES_FROZEN).filter(finance.STK_SHARES_FROZEN.code==code).limit(n))
# # 受限股份上市公告日期
# from jqdata import finance
# finance.run_query(query(finance.STK_LIMITED_SHARES_LIST).filter(finance.STK_LIMITED_SHARES_LIST.code==code).limit(n))
# # 受限股份实际解禁日期
# from jqdata import finance
# finance.run_query(query(finance.STK_LIMITED_SHARES_UNLIMIT).filter(finance.STK_LIMITED_SHARES_UNLIMIT.code==code).limit(n))
# # 上市公司股本变动
# from jqdata import finance
# finance.run_query(query(finance.STK_CAPITAL_CHANGE).filter(finance.STK_CAPITAL_CHANGE.code==code).limit(n))

# # 股东户数【最新数据不更新了】
# from jqdata import finance
# n=5000
# df=finance.run_query(query(finance.STK_HOLDER_NUM).limit(n))
# # df=finance.run_query(query(finance.STK_HOLDER_NUM).filter(finance.STK_HOLDER_NUM.code==code).limit(n))
# print(df)
# df.to_csv("股东户数.csv")

# # 宏观经济数据，stat_month【月度数据:'2014-02'】，stat_year【年度数据:'2014'】
# MAC_INDUSTRY_AREA_AGR_OUTPUT_VALUE_QUARTER分地区农林牧渔业总产值表(季度)
# MAC_INDUSTRY_AREA_AGR_OUTPUT_VALUE_YEAR分地区农林牧渔业总产值表(年度)
# MAC_INDUSTRY_AGR_PRODUCT_IDX_QUARTER全国农产品生产价格指数表(季度)
# MAC_SALE_RETAIL_MONTH社会消费品销售总额（月度）
# MAC_SALE_SCALE_RETAIL_MONTH限额以上零售分类表（月度）
# MAC_AREA_RETAIL_SALE分地区消费品零售总额（年度）
# MAC_SALE_MARKET亿元以上商品交易市场基本情况（年度）
# MAC_AREA_SALE_MARKET分地区亿元以上商品交易市场基本情况（年度）
# MAC_AREA_UNEMPLOY分地区城镇登记失业率（年度）
# MAC_EMPLOY_YEAR就业情况基本表(年度)
# MAC_AREA_WAGEIDX_YEAR分地区城镇单位就业人员情况表(年度)
# MAC_AREA_INDUSTRY_WAGE_YEAR分地区分行业城镇单位就业人员工资情况表(年度)
# MAC_INDUSTRY_WAGE_YEAR分行业城镇单位就业人员工资情况表(年度)
# MAC_RMB_EXCHANGE_RATE人民币外汇牌价(日级)
# MAC_LEND_RATE银行间拆借利率表（日级）
# MAC_MONEY_SUPPLY_MONTH货币供应量(月度)
# MAC_MONEY_SUPPLY_YEAR货币供应量(年度)
# MAC_CURRENCY_STATE_YEAR货币当局资产负债表（年度））
# MAC_SOCIAL_SCALE_FINANCE社会融资规模及构成（年度）
# MAC_STK_MARKET证券市场基本情况（年度）
# MAC_GOLD_FOREIGN_RESERVE黄金和外汇储备（月度）
# MAC_STK_ISSUE股票发行量和筹资额（年度）
# MAC_STK_TRADE股票市场统计表（年度）

# #查询2014年的分地区农林牧渔业总产值表(年度)
# df = macro.run_query(query(macro.MAC_INDUSTRY_AREA_AGR_OUTPUT_VALUE_YEAR).filter(macro.MAC_INDUSTRY_AREA_AGR_OUTPUT_VALUE_YEAR.stat_year=='2014'))
# print(df)
# # id	id	数字		
# # stat_quarter	统计季度	文本		YYYY-MM(03、06、09、12分别代表第1、2、3、4季度)
# # agricultural_products	农产品生产价格指数_当季值	数字		农产品生产价格指数是反映一定时期内，农产品生产者出售农产品价格水平变动趋势及幅度的相对数。该指数可以客观反映全国农产品生产价格水平和结构变动情况，满足农业与国民经济核算需要。其中某代表品生产价格指数是通过对全部有出售该产品行为的调查单位的个体指数进行几何平均求得的，类价格指数是通过对其所属的类（或代表品）的价格指数进行加权平均求得的。季度累计价格指数的计算方法与分季指数的计算方法相同
# # crop_products	种植业产品生产价格指数_当季值	数字	
# # total	农林牧渔业总产值	数字	亿元	农林牧渔业总产值指以货币表现的农、林、牧、渔业全部产品和对农林牧渔业生产活动进行的各种支持性服务活动的价值总量，它反映一定时期内农林牧渔业生产总规模和总成果。
# # farming	农业总产值	数字	亿元	
# # forestry	林业总产值	数字	亿元	
# # animal_husbandry	牧业总产值	数字	亿元	
# # fishery	渔业总产值	数字	亿元	
# # total_idx	农林牧渔业总产值指数	数字	%	上年=100，按可比价格
# # farming_idx	农业总产值指数	数字	%	上年=100
# # forestry_idx	林业总产值指数	数字	%	上年=100
# # animal_husbandry_idx	牧业总产值指数	数字	%	上年=100
# # fishery_idx	渔业总产值指数	数字	%	上年=100

# #查询2014年02月份的社会消费品销售总额（月度）
# df = macro.run_query(query(macro.MAC_SALE_RETAIL_MONTH).filter(macro.MAC_SALE_RETAIL_MONTH.stat_month=='2014-02'))
# print(df)

# #货币供应量
# df = macro.run_query(query(macro.MAC_MONEY_SUPPLY_MONTH))
# print(type(df["stat_month"].values[0]))
# df["stat_month"]=df["stat_month"].replace('-', '', regex=True).astype(float)
# df["stat_month"]=df["stat_month"]/1000000
# print(df)
# df=df.sort_values(by='stat_month')
# df.to_csv("货币供应量(月度)未来函数.csv")




# # 期权数据获取
# # import pandas as pd
# # import numpy as np
# # import talib as tb
# # from   pandas.plotting import  table
# # from   jqdata import *
# # #期权合约信息
# # class OptionContract:
# #     preClose: 0     #前一天的收盘价
# #     marginUnit: 0   #单位保证金
# #     prePosition: 0  #前一天收盘的持仓量
# #     close: 0        #现价
# #     volume: 0       #成交量
# #     position: 0     #当前持仓量
# #     priceChangePct: 0 #收盘价涨跌幅
# #     positionChange: 0 #日增仓
# #     positionChangePct: 0 #日增仓比例
# #     rewardPct: 0         #报酬率（权利金/保证金）
# #     def __init__(self, preClose, marginUnit, 
# #                  prePosition, close, volume, 
# #                  position, priceChangePct, 
# #                  positionChange, 
# #                  positionChangePct, 
# #                  rewardPct):
# #         self.preClose = preClose
# #         self.marginUnit = marginUnit
# #         self.prePosition = prePosition
# #         self.close = close
# #         self.volume = volume
# #         self.position = position
# #         self.priceChangePct = priceChangePct
# #         self.positionChange = positionChange
# #         self.positionChangePct = positionChangePct
# #         self.rewardPct = rewardPct
# # # 同一行权价的一组期权
# # class OptionGroup:
# #     exercisePrice: 0 ##行权价
# #     call: None       ##认购的类型
# #     put: None        ##认沽的类型  
# #     def __init__(self, exercisePrice, call, put):
# #         self.exercisePrice = exercisePrice
# #         self.call = call
# #         self.put = put
# #     def __init__(self, exercisePrice):
# #         self.exercisePrice = exercisePrice

# # #数据实体
# # CONTENT = [
# #     'close', 
# #     'volume', 
# #     'position', 
# #     'priceChangePct', 
# #     'positionChange', 
# #     'positionChangePct',
# #     'preClose', 
# #     'marginUnit', 
# #     'rewardPct'
# # ]
# # #目前支持的域及对应的抬头
# # CONTENT_HEADER_MAP = {
# #     'rewardPct': '报酬率', #计算方式：权利金/卖出一手所需的保证金*100
# #     'marginUnit': '保证金', #开盘前一手该期权的保证金额，没有随盘中价格变化更新
# #     'preClose': '昨收',
# #     'prePosition': '昨日持仓',
# #     'close': '现价', 
# #     'volume': '成交量', 
# #     'position': '持仓量', 
# #     'priceChangePct': '涨跌幅', 
# #     'positionChange': '日增仓', 
# #     'positionChangePct': '日增仓率', 
# # }
# # #期权合约，以50ETF为例 
# # SUBJECT_MATTER = '510050.XSHG'
# # #期权合约，以300ETF为例 
# # #SUBJECT_MATTER = '510300.XSHG'
# # #查询数据日期，以2024-03-22为例
# # DATA_DATE = '2024-03-22'
# # #合约到期日，以50ETF2403为例
# # EXPIRE_MONTH = '2024-03-27'
# # #数据实体
# # CONTENT = [
# #     'close', 
# #     'volume', 
# #     'position', 
# #     'priceChangePct', 
# #     'positionChange', 
# #     'positionChangePct',
# #     'preClose', 
# #     'marginUnit', 
# #     'rewardPct'
# # ]
# # ##目前支持的域及对应的抬头
# # CONTENT_HEADER_MAP = {
# #     'rewardPct': '报酬率', #计算方式：权利金/卖出一手所需的保证金*100
# #     'marginUnit': '保证金', #开盘前一手该期权的保证金额，没有随盘中价格变化更新
# #     'preClose': '昨收',
# #     'prePosition': '昨日持仓',
# #     'close': '现价', 
# #     'volume': '成交量', 
# #     'position': '持仓量', 
# #     'priceChangePct': '涨跌幅', 
# #     'positionChange': '日增仓', 
# #     'positionChangePct': '日增仓率', 
# # }
# # #期权合约，以50ETF为例 
# # SUBJECT_MATTER = '510050.XSHG'
# # #期权合约，以300ETF为例 
# # #SUBJECT_MATTER = '510300.XSHG'
# # #查询数据日期，以2024-03-22为例
# # DATA_DATE = '2024-03-22'
# # #合约到期日，以50ETF2403为例
# # EXPIRE_MONTH = '2024-03-27'
# # #查询相关的合约
# # qy = query(opt.OPT_CONTRACT_INFO).filter(
# #     opt.OPT_CONTRACT_INFO.underlying_symbol == SUBJECT_MATTER, ##期权标的物
# #     opt.OPT_CONTRACT_INFO.exercise_date == EXPIRE_MONTH,##期权到期日
# # ).order_by(opt.OPT_CONTRACT_INFO.exercise_price, opt.OPT_CONTRACT_INFO.contract_type)
# # optList = opt.run_query(qy)
# # print(optList)
# # optionGroups = {}
# # for index, row in optList.iterrows():
# #     code = row['code'] #key - 期权代码
# #     #查询具体合约的信息,商品需要去掉开盘前的静态信息
# #     optionDailyQuery = query(opt.OPT_DAILY_PREOPEN).filter(
# #                              opt.OPT_DAILY_PREOPEN.code==code).order_by(
# #                              opt.OPT_DAILY_PREOPEN.date.desc()).limit(1)
# #     dailyData = opt.run_query(optionDailyQuery)
# #     realTimeQuery = query(opt.OPT_DAILY_PRICE).filter(
# #                           opt.OPT_DAILY_PRICE.code==code, 
# #                           opt.OPT_DAILY_PRICE.date==DATA_DATE).order_by(
# #                           opt.OPT_DAILY_PRICE.date.desc()).limit(1)
# #     realTimeData = opt.run_query(realTimeQuery)
# #     #期权基本信息
# #     exercisePrice = row['exercise_price'] #行权价
# #     contractType = row['contract_type'] #合约类型。CO-认购期权，PO-认沽期权
# #     #盘前静态表查询
# #     preClose = dailyData.loc[0].at['pre_close'] #前一天的收盘价
# #     marginUnit = int(dailyData.loc[0].at['margin_unit']) #单位保证金
# #     prePosition = dailyData.loc[0].at['position'] #前一天收盘的持仓量
# #     #实时表查询
# #     close = realTimeData.loc[0].at['close'] #现价
# #     volume = int(realTimeData.loc[0].at['volume']) #成交量
# #     position = realTimeData.loc[0].at['position'] #当前持仓量
# #     priceChangePct = str(round(realTimeData.loc[0].at['change_pct_close'], 2)) + '%' #收盘价涨跌幅
# #     #计算
# #     positionChange = position - prePosition #日增仓
# #     positionChangePct = str(round(abs(positionChange / prePosition * 100), 2)) + '%' #日增仓比例
# #     rewardPct = round(close * 10000 / marginUnit * 100, 2) #报酬率（权利金/保证金）
# #     #去除非标准的带A合约
# #     if(exercisePrice * 10000 % 500 != 0):
# #         continue
# #     optionContract = OptionContract(preClose, marginUnit, 
# #                                     prePosition, close, 
# #                                     volume, position, 
# #                                     priceChangePct, positionChange, 
# #                                     positionChangePct, rewardPct)
# #     if(exercisePrice in optionGroups):
# #         if(contractType == 'CO'):
# #             optionGroups[exercisePrice].call = optionContract
# #         else:
# #             optionGroups[exercisePrice].put = optionContract
# #     else:
# #         optionGroup = OptionGroup(exercisePrice)
# #         if(contractType == 'CO'):
# #             optionGroup.call = optionContract
# #         else:
# #             optionGroup.put = optionContract
# #         optionGroups[exercisePrice] = optionGroup
# # list = optionGroups.values()
# # rtitle = CONTENT.copy()
# # rtitle.reverse()
# # data = {}
# # for key in rtitle:
# #     data['C-'+CONTENT_HEADER_MAP[key]] = []
# # data['行权价'] = []
# # for key in CONTENT:
# #     data['P-'+CONTENT_HEADER_MAP[key]] = []
# # for option in list:
# #     for key in CONTENT:
# #         data['C-'+CONTENT_HEADER_MAP[key]].append(getattr(option.call, key))
# #     data['行权价'].append(option.exercisePrice)
# #     for key in rtitle:
# #         data['P-'+CONTENT_HEADER_MAP[key]].append(getattr(option.put, key))
# # df = pd.DataFrame(data)
# # print(df)








# #备兑期权【持有现货卖出看涨期权】回测
# import jqdata
# from jqdata import *
# import pandas as pd
# import numpy as np
# import datetime
# import matplotlib.pyplot as plt
# plt.rcParams['font.sans-serif']=['SimHei']
# plt.rcParams['axes.unicode_minus'] = False
# #获取交易时间和时间间隔（频率：月）
# # targetetf="510050.XSHG"
# targetetflist=[
#             "159901.XSHE","159915.XSHE","159919.XSHE","159922.XSHE",
#             "510050.XSHG","510300.XSHG",
#             "510500.XSHG","588000.XSHG","588080.XSHG",
#             ]#沪深300和上证50比较好
# for targetetf in targetetflist:
#     startyear="2020"
#     endyear="2023"
#     trade_days = pd.Series(index=jqdata.get_trade_days(f'{startyear}-09-01',f'{endyear}-11-10'))
#     trade_days.index = pd.to_datetime(trade_days.index)
#     month_split = list(trade_days.resample('M',label='left').mean().index) + [pd.to_datetime(f'{endyear}1130')]
#     print("month_split",month_split)
#     #传统备兑开仓策略
#     holding_contract2 = pd.Series(index=trade_days.index)
#     #获取首个持仓合约
#     q_contract_info = query(opt.OPT_CONTRACT_INFO.code, opt.OPT_CONTRACT_INFO.trading_code, opt.OPT_CONTRACT_INFO.name, #合约代码,合约交易代码,合约简称
#               opt.OPT_CONTRACT_INFO.exercise_price, opt.OPT_CONTRACT_INFO.last_trade_date, 
#               opt.OPT_CONTRACT_INFO.list_date
#              ).filter(opt.OPT_CONTRACT_INFO.contract_type == 'CO',  #看涨期权
#                       opt.OPT_CONTRACT_INFO.exchange_code == 'XSHG',   #上交所
#                      opt.OPT_CONTRACT_INFO.last_trade_date > month_split[0],  #时间-到期月开始
#                      opt.OPT_CONTRACT_INFO.last_trade_date < month_split[1],  #时间-到期月结束
#                      opt.OPT_CONTRACT_INFO.list_date < trade_days.index[0])    #在交易前上市
#     contract_info = opt.run_query(q_contract_info)
#     etf_cls = get_price(targetetf,trade_days.index[0],trade_days.index[0],fields=['close']).values[0][0]
#     contract_info['price_spread'] = contract_info['exercise_price'] - etf_cls
#     if contract_info['price_spread'].max() > 0:
#         contract_info = contract_info[contract_info['price_spread'] > 0]  #选出虚值期权
#         contract_info = contract_info.sort_values('exercise_price')
#     else:  #全是实值期权
#         contract_info = contract_info.sort_values('exercise_price',ascending=False)    
#     holding_contract2[trade_days.index[0]] = contract_info['code'].iloc[0]
#     newest_exercise_price = contract_info['exercise_price'].iloc[0]
#     newest_expire_date = contract_info['last_trade_date'].iloc[0]
#     #逐日判断对应的交易合约
#     #规则:持有略虚值看涨期权，待行权价低于现价的95%时，平仓原期权合约，重新开仓略虚值看涨期权；到期前1天移仓换月至次月合约
#     for t in trade_days.index[1:]:
#         #到期前一天
#         if t >= pd.to_datetime(get_trade_days(end_date=pd.to_datetime(newest_expire_date),count=2)[0]):
#             #寻找month_idx
#             for month_idx in range(len(month_split)):
#                 if month_split[month_idx] >= t:
#                     break  
#             q_contract_info = query(opt.OPT_CONTRACT_INFO.code, opt.OPT_CONTRACT_INFO.trading_code, opt.OPT_CONTRACT_INFO.name, 
#                       opt.OPT_CONTRACT_INFO.exercise_price, opt.OPT_CONTRACT_INFO.last_trade_date,  
#                       opt.OPT_CONTRACT_INFO.list_date
#                      ).filter(opt.OPT_CONTRACT_INFO.contract_type == 'CO',  #看涨期权
#                               opt.OPT_CONTRACT_INFO.exchange_code == 'XSHG',   #上交所
#                              opt.OPT_CONTRACT_INFO.last_trade_date > month_split[month_idx],  #时间-到期月开始
#                              opt.OPT_CONTRACT_INFO.last_trade_date <= month_split[month_idx+1],  #时间-到期月结束
#                              opt.OPT_CONTRACT_INFO.list_date < t)    #在交易前上市
#             contract_info = opt.run_query(q_contract_info)
#             etf_cls = get_price('510050.XSHG',t,t,fields=['close']).values[0][0]   #现货收盘价
#             contract_info['price_spread'] = contract_info['exercise_price'] - etf_cls
#             if contract_info['price_spread'].max() > 0:
#                 contract_info = contract_info[contract_info['price_spread'] > 0]  #选出虚值期权
#                 contract_info = contract_info.sort_values('exercise_price')
#             else:  #全是实值期权
#                 contract_info = contract_info.sort_values('exercise_price',ascending=False) 
#             if contract_info['last_trade_date'].iloc[0] >= newest_expire_date:
#                 holding_contract2[t] = contract_info['code'].iloc[0]
#                 newest_exercise_price = contract_info['exercise_price'].iloc[0]
#                 newest_expire_date = contract_info['last_trade_date'].iloc[0]
#         else:
#             #获取昨日targetetf收盘价
#             pre_cls = get_price(targetetf,t,t,fields=['pre_close']).values[0][0]
#             if pre_cls*0.95 >= newest_exercise_price:  #原虚值变为实值,重新开仓略虚值期权
#                 #寻找month_idx
#                 for month_idx in range(len(month_split)):
#                     if month_split[month_idx] >= t:
#                         break
#                 q_contract_info = query(opt.OPT_CONTRACT_INFO.code, opt.OPT_CONTRACT_INFO.trading_code, opt.OPT_CONTRACT_INFO.name, 
#                           opt.OPT_CONTRACT_INFO.exercise_price, opt.OPT_CONTRACT_INFO.last_trade_date,  #行权价格,最后交易日
#                           opt.OPT_CONTRACT_INFO.list_date
#                          ).filter(opt.OPT_CONTRACT_INFO.contract_type == 'CO',  #看涨期权
#                                   opt.OPT_CONTRACT_INFO.exchange_code == 'XSHG',   #上交所
#                                  opt.OPT_CONTRACT_INFO.last_trade_date > month_split[month_idx-1],  #时间-到期月开始
#                                  opt.OPT_CONTRACT_INFO.last_trade_date <= month_split[month_idx],  #时间-到期月结束
#                                  opt.OPT_CONTRACT_INFO.list_date < t)    #在交易前上市
#                 contract_info = opt.run_query(q_contract_info)
#                 etf_cls = get_price(targetetf,t,t,fields=['close']).values[0][0]
#                 contract_info['price_spread'] = contract_info['exercise_price'] - etf_cls
#                 if contract_info['price_spread'].max() > 0: 
#                     contract_info = contract_info[contract_info['price_spread'] > 0]  #选出虚值期权
#                     contract_info = contract_info.sort_values('exercise_price')
#                 else:  #全是实值期权
#                     contract_info = contract_info.sort_values('exercise_price',ascending=False) 
#                 if contract_info['last_trade_date'].iloc[0] >= newest_expire_date:
#                     holding_contract2[t] = contract_info['code'].iloc[0]
#                     newest_exercise_price = contract_info['exercise_price'].iloc[0]  
#                     newest_expire_date = contract_info['last_trade_date'].iloc[0]
#     holding_contract2 = holding_contract2.fillna(method='ffill')
#     print(holding_contract2)
#     data2 = pd.DataFrame(holding_contract2)
#     data2.columns = ['holding_contract']
#     data2 = data2.reindex(columns=['holding_contract','close','last_close'])
#     last_contract = holding_contract2.iloc[0]   #记录上个持仓
#     for t in data2.index:
#         if last_contract == data2.loc[t,'holding_contract']:  #期权未换仓
#             #收盘价
#             q_price = query(opt.OPT_DAILY_PRICE.code, opt.OPT_DAILY_PRICE.date, opt.OPT_DAILY_PRICE.close,
#                      ).filter(opt.OPT_DAILY_PRICE.code==last_contract,
#                              opt.OPT_DAILY_PRICE.date==t)
#             price = opt.run_query(q_price)['close'][0]
#             data2.loc[t,'close'] = price
#         else:
#             #收盘价,新
#             q_price = query(opt.OPT_DAILY_PRICE.code, opt.OPT_DAILY_PRICE.date, opt.OPT_DAILY_PRICE.close,
#                      ).filter(opt.OPT_DAILY_PRICE.code==data2.loc[t,'holding_contract'],
#                              opt.OPT_DAILY_PRICE.date==t)
#             price = opt.run_query(q_price)['close'][0]
#             data2.loc[t,'close'] = price
#             #收盘价,旧
#             q_price = query(opt.OPT_DAILY_PRICE.code, opt.OPT_DAILY_PRICE.date, opt.OPT_DAILY_PRICE.close,
#                      ).filter(opt.OPT_DAILY_PRICE.code==last_contract,
#                              opt.OPT_DAILY_PRICE.date==t)
#             price = opt.run_query(q_price)['close'][0]
#             data2.loc[t,'last_close'] = price
#             last_contract = data2.loc[t,'holding_contract']
#     print(data2)
#     #卖出期权收益（值）
#     opt_ret2 = pd.Series(0,index=data2.index)
#     pre_close2 = data2['close'].iloc[0]
#     for t in data2.index[1:]:
#         if data2.isna().loc[t,'last_close']:   #未换仓,last为空
#             opt_ret2[t] = -10000*(data2.loc[t,'close'] - pre_close2)
#         else:
#             opt_ret2[t] = -10000*(data2.loc[t,'last_close'] - pre_close2) - 5  #手续费5元
#         pre_close2 = data2.loc[t,'close']
#     print(opt_ret2)
#     #targetetf持仓收益（值）
#     etf_price = get_price(targetetf,trade_days.index[0],trade_days.index[-1],fields=['close'])['close']
#     etf_ret = 10000*etf_price.diff(1).fillna(0)
#     print(etf_ret)
#     init_asset2 = etf_price.iloc[0]*10000
#     ass2 = init_asset2 + (etf_ret + opt_ret2).cumsum()
#     pfl_ret2 = (ass2/ass2.shift(1) - 1).fillna(0)
#     pfl_nv2 = (1 + pfl_ret2).cumprod()
#     print(pfl_nv2)
#     #画图
#     plt.figure(figsize=(8, 5))
#     plt.plot(etf_price/etf_price.iloc[0], label=f'{targetetf}ETF现货净值')
#     plt.plot(pfl_nv2, label='备兑策略净值')
#     plt.legend(loc='upper left', fontsize='large')
#     plt.xlabel('时间',size=12)
#     plt.ylabel('净值',size=12)
#     plt.show()

      订单状态 comment         收益 cancel_time  ...         标的代码    委托方式  side     已成交量
0     done                0.0        None  ...  600733.XSHG  market  long    71400
1     done                0.0        None  ...  000736.XSHE  market  long    90900
2     done                0.0        None  ...  000035.XSHE  market  long   125000
3     done                0.0        None  ...  600234.XSHG  market  long   139600
4     done                0.0        None  ...  600519.XSHG  market  long     5200
...    ...     ...        ...         ...  ...          ...     ...   ...      ...
2852  done                0.0        None  ...  002287.XSHE  market  long   336000
2853  done          1060756.0        None  ...  002223.XSHE  market  long   208400
2854  done          1346695.0        None  ...  002595.XSHE  market  long   247100
2855  done            23520.0        None  ...  002287.XSHE  market  long   336000
2856  done          -259850.0        None  ...  002086.XSHE  market  long  2598500

[28

         id         day  name  ... a_quote_change h_quote_change  h_a_comp
0    454068  2020-12-28   万科A  ...         1.1388           0.78    1.3121
1    454025  2020-12-28  中集集团  ...         1.1061           2.95    1.5577
2    454108  2020-12-28  中兴通讯  ...        -1.5142          -1.42    2.0145
3    454058  2020-12-28  中联重科  ...         2.4811           1.35    1.2521
4    454064  2020-12-28  申万宏源  ...         0.0000           1.00    2.9897
..      ...         ...   ...  ...            ...            ...       ...
125  454116  2020-12-28  君实生物  ...        -0.0553          -1.65    1.6020
126  454125  2020-12-28   康希诺  ...        -7.8340          -2.66    2.6494
127  454123  2020-12-28  昊海生科  ...        -3.4968          -2.15    2.2513
128  454129  2020-12-28  复旦张江  ...        -0.0573          -3.81    5.4736
129  785589  2020-12-28  中芯国际  ...        -2.6203          -4.20    3.3911

[130 rows x 10 columns]
2020-12-29
         id         day   name  ... a_quote_change h_quote_chang

         id         day  name  ... a_quote_change h_quote_change  h_a_comp
0    455299  2021-01-11   万科A  ...         1.4997           5.39    1.2627
1    455387  2021-01-11  中集集团  ...        -5.6569          -2.67    1.4189
2    455327  2021-01-11  中兴通讯  ...         3.0383           0.94    1.9529
3    455378  2021-01-11  中联重科  ...        -3.6256          -1.70    1.3211
4    455379  2021-01-11  申万宏源  ...        -2.8571          -1.81    2.8251
..      ...         ...   ...  ...            ...            ...       ...
125  455409  2021-01-11  君实生物  ...        -1.6517          -5.00    1.7017
126  455414  2021-01-11   康希诺  ...        -6.6269          -6.36    2.1007
127  455408  2021-01-11  昊海生科  ...        -0.2494          -4.94    2.0343
128  455373  2021-01-11  复旦张江  ...        -2.8888          -1.52    4.8950
129  785598  2021-01-11  中芯国际  ...         1.8022           1.20    2.9790

[130 rows x 10 columns]
2021-01-12
         id         day  name  ... a_quote_change h_quote_change

2021-01-26
         id         day  name  ... a_quote_change h_quote_change  h_a_comp
0    459417  2021-01-26   万科A  ...         1.1580           1.53    1.1918
1    460622  2021-01-26  中集集团  ...        -5.7909          -3.85    1.4514
2    461826  2021-01-26  中兴通讯  ...        -4.6779          -4.79    1.7839
3    463031  2021-01-26  中联重科  ...        -5.1682          -4.00    1.3112
4    463459  2021-01-26  申万宏源  ...        -1.6632          -3.81    2.4959
..      ...         ...   ...  ...            ...            ...       ...
125  583593  2021-01-26  君实生物  ...        -0.2360           0.00    1.2377
126  583705  2021-01-26   康希诺  ...        -0.5152          -1.11    1.6719
127  584010  2021-01-26  昊海生科  ...        -3.0253          -1.29    1.9507
128  584159  2021-01-26  复旦张江  ...        -2.1146          -2.08    4.0596
129  785609  2021-01-26  中芯国际  ...         3.0334           0.50    2.3750

[130 rows x 10 columns]
2021-01-27
         id         day  name  ... a_quote_change h_q

2021-02-08
         id         day  name  ... a_quote_change h_quote_change  h_a_comp
0    702903  2021-02-08   万科A  ...         0.8657           1.55    1.1854
1    702904  2021-02-08  中集集团  ...         0.6881          -1.85    1.3536
2    702905  2021-02-08  中兴通讯  ...         0.0971          -1.42    1.7760
3    702906  2021-02-08  中联重科  ...        10.0000           8.88    1.3248
4    702907  2021-02-08  申万宏源  ...         0.2165           1.88    2.5613
..      ...         ...   ...  ...            ...            ...       ...
126  703029  2021-02-08  君实生物  ...         4.4958          -3.05    1.3881
127  703030  2021-02-08   康希诺  ...         3.9677           8.89    1.3711
128  703031  2021-02-08  昊海生科  ...         0.5246           2.73    2.2634
129  703032  2021-02-08  复旦张江  ...         0.1337          -0.64    3.8425
130  703033  2021-02-08  中芯国际  ...        -0.0960          -3.28    2.6456

[131 rows x 10 columns]
2021-02-09
         id         day  name  ... a_quote_change h_q

...

2023-05-30
         id  company_id  ... price_ceiling after_change_ratio
0    387495   300003427  ...   21.42-24.21             12.288
1    387501   300010150  ...          None              0.000
2    387502   300010389  ...         11.79                NaN
3    387503   300010389  ...         11.01              0.000
4    387504   300014607  ...          None              5.870
..      ...         ...  ...           ...                ...
159  387727   300037230  ...         19.08              2.730
160  387730   440834903  ...          None              5.168
161  387731   440839473  ...         73.91              1.490
162  387734   440871269  ...          None              2.835
163  387735   440871269  ...         63.86              5.880

[164 rows x 13 columns]
2023-05-31
        id  company_id  ... price_ceiling after_change_ratio
0   387643   300004301  ...          5.17              2.332
1   387644   300007908  ...          None              5.000
2   387646   300012290  ..

2023-06-07
         id  company_id  ... price_ceiling after_change_ratio
0    388736   300001463  ...         7.421             24.040
1    388737   300006137  ...          None              6.920
2    388738   300006641  ...          None              8.042
3    388739   300007969  ...          None                NaN
4    388740   300007969  ...          None                NaN
..      ...         ...  ...           ...                ...
346  389318   300039374  ...          40.8              5.000
347  389319   300041484  ...         68.43              0.140
348  389332   440831517  ...         14.48              0.201
349  389333   440831517  ...         13.80              0.188
350  389342   440839243  ...         54.32              0.201

[351 rows x 13 columns]
2023-06-08
         id  company_id  ... price_ceiling after_change_ratio
0    389233   300001762  ...          3.40             10.720
1    389234   300004262  ...          None                NaN
2    389235   300004262

2023-06-26
        id  company_id  ... price_ceiling after_change_ratio
0   391786   300000341  ...          None              6.029
1   391787   300003421  ...         37.14                NaN
2   391788   300003421  ...         39.35              0.104
3   391789   300007107  ...          None              0.000
4   391803   300012131  ...         23.25                NaN
..     ...         ...  ...           ...                ...
89  392150   300041876  ...        105.00                NaN
90  392152   300041876  ...         95.54              7.170
91  392155   440830970  ...         10.27              4.730
92  392158   440871785  ...         20.38              0.340
93  392159   440871785  ...         20.38              0.800

[94 rows x 13 columns]
2023-06-27
        id  company_id  ... price_ceiling after_change_ratio
0   392060   300000123  ...          9.81                NaN
1   392061   300000123  ...          9.85                NaN
2   392062   300000123  ...         10.

2023-07-06
         id  company_id  ... price_ceiling after_change_ratio
0    393953   300004546  ...          None                NaN
1    393954   300004546  ...          None                NaN
2    393955   300004546  ...          None                NaN
3    393956   300004546  ...          None                NaN
4    393957   300004546  ...          None                NaN
..      ...         ...  ...           ...                ...
241  394401   440871269  ...         54.77                NaN
242  394402   440871269  ...         59.49                NaN
243  394403   440871269  ...         58.57              9.939
244  394415   361000381  ...          None              9.555
245  394416   361000381  ...          None              9.433

[246 rows x 13 columns]
2023-07-07
         id  company_id  ... price_ceiling after_change_ratio
0    394294   300002002  ...          None                NaN
1    394295   300002002  ...          None                NaN
2    394296   300002002

2023-07-18
         id  company_id  ... price_ceiling after_change_ratio
0    395834   300002702  ...          7.85                NaN
1    395835   300002702  ...          8.42                NaN
2    395836   300002702  ...          8.42                NaN
3    395837   300002702  ...          8.42                NaN
4    395838   300002702  ...          8.43              0.760
..      ...         ...  ...           ...                ...
183  396196   200000034  ...         33.45              0.105
184  396200   440834991  ...         20.14              6.160
185  396203   440835666  ...       31.9467                NaN
186  396205   440835666  ...       31.9467              4.964
187  396206   440837048  ...          None              8.940

[188 rows x 13 columns]
2023-07-19
        id  company_id  ... price_ceiling after_change_ratio
0   396129   300010349  ...          5.99              0.000
1   396133   300020287  ...     7.32-7.59             50.000
2   396134   300021171  ..